In [1]:
import sys
sys.path.insert(0, "scripts")

start = datetime.date(2021, 1, 1)
end = datetime.date(2021, 2,1)
lb = load_bars(start, end, 1)
lb.tail()

ModuleNotFoundError: 'pyarrow.parquet' is required when using `read_parquet(..., use_pyarrow=True)`.
Please install using the command `pip install pyarrow`.

In [1]:
import tushare as ts
pro = ts.pro_api()

# 调取单只股票盈利预测（2024-01-01 至 2024-12-31）
df = pro.report_rc(ts_code="600000.SH", start_date="20240101", end_date="20241231")
print(df)

2025-12-31 10:54:37.948 | INFO     | tushare:_connect:30 - Connected to server tushare:5290


       ts_code  name report_date               report_title report_type  \
0    600000.SH  浦发银行    20241225          浦发银行：改革见效，基本面逐步夯实          一般   
1    600000.SH  浦发银行    20241225          浦发银行：改革见效，基本面逐步夯实          一般   
2    600000.SH  浦发银行    20241220        浦发银行：股东增持显信心，增长动能强劲          点评   
3    600000.SH  浦发银行    20241220        浦发银行：股东增持显信心，增长动能强劲          点评   
4    600000.SH  浦发银行    20241220        浦发银行：股东增持显信心，增长动能强劲          点评   
..         ...   ...         ...                        ...         ...   
121  600000.SH  浦发银行    20240320          浦发银行：业绩筑底，发力五篇大文章          点评   
122  600000.SH  浦发银行    20240320          浦发银行：业绩筑底，发力五篇大文章          点评   
123  600000.SH  浦发银行    20240320  浦发银行2023年业绩快报点评：业绩筑底，动能改善          点评   
124  600000.SH  浦发银行    20240320  浦发银行2023年业绩快报点评：业绩筑底，动能改善          点评   
125  600000.SH  浦发银行    20240320  浦发银行2023年业绩快报点评：业绩筑底，动能改善          点评   

    classify org_name      author_name quarter       op_rt  ...         tp  \
0       一般报告       中金

In [2]:
from pathlib import Path
import re

path = Path("~/workspace/zillionare/max-dama.srt").expanduser()
with open(path, "r", encoding="utf-8") as f:
    lines = f.readlines()

timestamps = ["00:00:00.000"]
for i in range(len(lines)):
    if re.match(r"^\d+:\d+", lines[i]):
        time = lines[i].split(":")
        mm = int(time[0])
        ss = int(time[1])
        timestamps.append(f"00:{mm:02d}:{ss:02d}.000")

timestamps.append("00:54:55,000")

print(len(lines), len(timestamps))
j = 0
m = 0
buffers = []
for i in range(len(lines)):
    line = lines[i]
    if re.match(r"^\d+:\d+", line):
        j += 1
        buffers.append(f"\n{j}\n")
        line = timestamps[j] + " --> " + timestamps[j+1] + "\n"
        buffers.append(line)
        continue
    
    buffers.append(line)
    if len(buffers) > 100:
        m += 1
        to = path.parent / f"srt/{m:02d}.srt"
        with open(to, "w", encoding="utf-8") as f:
            f.writelines(buffers)
            buffers = []

to = path.parent / f"srt/{m:02d}.srt"
if len(buffers) > 0:
    with open(to, "w", encoding="utf-8") as f:
        f.writelines(buffers)

1085 541


In [27]:
import sqlite_utils as su

db = su.Database(":memory:")

db["users"].insert_all([{"id": 1, "name": "Fred"}, {"id": 2, "name": "Wilma"}])
list(db["users"].rows)

[{'id': 1, 'name': 'Fred'}, {'id': 2, 'name': 'Wilma'}]

In [14]:
db["users"].insert({"id": 3, "name": "Barney", "age": 10}, alter=True)

pd.DataFrame(db["users"].rows)

,id,name,age
0,1,Fred,NaN
1,2,Wilma,NaN
2,3,Barney,10.0


In [59]:
from dataclasses import dataclass, asdict, fields
import types
import sqlite_utils as su
from enum import IntEnum
import datetime

def _dataclass_to_schema(model) -> dict:
    """类方法：解析当前 dataclass 为 fastlite 兼容的 schema 字典"""
    schema = {}

    for f in fields(model):
        if f.type in (str, int, float, bool):
            schema[f.name] = f.type
        # 处理所有联合类型（Union[A, B] 和 A | B 语法）
        elif (
            hasattr(f.type, "__origin__") and f.type.__origin__ is Union
        ) or isinstance(f.type, types.UnionType):
            # 提取非 None 的类型
            non_none_types = [t for t in f.type.__args__ if t is not type(None)]
            if non_none_types:
                base_type = non_none_types[0]
                schema[f.name] = (
                    base_type if base_type in (str, int, float, bool) else str
                )
            else:
                schema[f.name] = str
        elif isinstance(f.type, type) and issubclass(f.type, IntEnum):
            schema[f.name] = int
        else:
            schema[f.name] = str
    return schema

def create_tables(db: su.Database, model):
    """初始化表结构
    
    在 sqlite_utils 中，创建表结构并非必须；但会导致sqlite-utils 无法准确判断类型。
    """
    table = model.__table__
    pk = model.__pk__

    t: su.db.Table = db[table] # type: ignore
    t.create(_dataclass_to_schema(model), pk=pk)

    if model.__indices__ is not None:
        indexes, is_unique = model.__indices__
        t.create_index(indexes, unique=is_unique)

class Gender(IntEnum):
    MALE = 1
    FEMALE = 2

@dataclass
class User:
    __table__ = "users"
    __pk__ = "id"
    __indices__ = (["name"], False)
    id: int
    name: str
    birth: datetime.date
    gender: Gender

    def __post_init__(self):
        if isinstance(self.birth, str):
            self.birth = datetime.date.fromisoformat(self.birth)
        if isinstance(self.gender, int):
            self.gender = Gender(self.gender)

In [ ]:
db = su.Database(memory=True)

create_tables(db, User)

user = User(id=1, 
            name="Alice", 
            birth=datetime.date(2020, 1, 1), 
            gender=Gender.FEMALE)

# 增加一条记录
db["users"].insert(asdict(user))

# 增加记录，如果存在，则进行更新
db["users"].upsert(asdict(user), pk="id")

# 上述语句相当于
db["users"].update(1, {"birth": datetime.date(2020, 1, 20)})

# 通过主键查询刚修改的记录，生日将变为2020-01-20
db["users"].get(1)

# 返回所有记录
list(db["users"].rows)

# 返回所有记录，转换为 DataFrame。这里 pandas/polars 都支持
pd.DataFrame(db["users"].rows)

# 按条件查询，使用 rows_where方法
list(db["users"].rows_where("gender = ?", (Gender.FEMALE,)))

# 当然，我们也可以使用 db.query 方法,显然不如上一种方法简洁
list(db.query("SELECT * FROM users WHERE gender = ?", (Gender.FEMALE,)))

# 按条件删除记录，使用 delete_where 方法
db["users"].delete_where("id = ?", (10,))

# 按id 删除，使用 delete 方法
db["users"].delete(1)

# 查询数据库中的记录数，此时应该为0
db["users"].count

# 删除user 表，使用 drop 方法
db["users"].drop()

# 确认表已删除，使用 tables属性。此时应该返回空列表
db.tables


[]

In [48]:
user_from_db = User(**list(db["users"].rows)[0])
user_from_db

User(id=1, name='Alice', birth=datetime.date(2020, 1, 1), gender=<Gender.FEMALE: 2>)

In [39]:
list(db["users"].rows)[0]

{'id': 1, 'name': 'Alice', 'birth': '2020-01-01', 'gender': 2}